In [1]:
%%html
<style> 
body {
  counter-reset: section subsection subsubsection;
}

h1 {
  counter-reset: section subsection subsubsection;
}

h2 {
  counter-reset: subsection subsubsection;
}

h2:before {
    counter-increment: section;
    content: counter(section) ". ";
}
    
h3 {
  counter-reset: subsubsection;
}

h3:before {
    counter-increment: subsection;
    content: counter(section) "." counter(subsection) " ";
}
    
h4:before {
    counter-increment: subsubsection;
    content: counter(section) "." counter(subsection) "." counter(subsubsection) " ";
}
</style>

<a id='TOP'></a>
# GxB2 Stats Explanation

1. [List of stats](#LIST)
1. [Calculation rules](#RULE)
1. [Order of buffs](#ORDER)
1. [Damage formula](#FORMULA)

<a id='LIST'></a>
## List of stats

There's a [complete list](https://github.com/afknst/gxb2_tables/blob/master/tables/buff.csv) that might be initially migrated from IH. Some stats are actually irrelevant.  
The stats that will be discussed here are listed below.

- The stats that show on the girl page:
    - `Attack`
    - `HP`
    - `Armor`
    - `Speed`
    - `Precision`
    - `Block`
    - `Crit`
    - `Crit_Resist`
    - `Crit_DMG`
    - `Armor_Brk`
    - `Skill_DMG`
    - `Holy_DMG`
    - `Control_Rate`
    - `CC_Immune`
    - `DMG_Red`
    - `DMG_Dep`
- The stats that show in equipment and lab:
    - `Energy`
    - `Heal_Received`
    - `Heal_Amount`
    - `Warrior_Red`
    - `Mage_Red`
    - `Ranger_Red`
    - `Assassin_Red`
    - `Priest_Red`
    - `DMG_Warrior`
    - `DMG_Mage`
    - `DMG_Ranger`
    - `DMG_Assassin`
    - `DMG_Priest`
    
Note that some "buffs" shown in equipment are actually implemented using "skills".  
Those won't be discussed here (e.g. Sacred Vial, Burning Blades, etc.).

<a id='RULE'></a>
## Calculation rules
1. [Additive rule](#ADD)
1. [Multiplicative rule](#MUL)
1. [Mixed rule](#MIXED)
1. [Bounds of stats](#BOUND)

<a id='ADD'></a>
### Additive rule

All stats except those mentioned in [Multiplicative rule](#MUL) and [Mixed rule](#MIXED) are using the additive rule, e.g.  
>A girl initially has 20% Block, then she receives a buff that adds 30% Block. She has 50% Block now.

<a id='MUL'></a>
### Multiplicative rule

The stats below are using the multiplicative rule:
- `Heal_Received`
- `Heal_Amount`
- `Warrior_Red`
- `Mage_Red`
- `Ranger_Red`
- `Assassin_Red`
- `Priest_Red`
- `DMG_Warrior`
- `DMG_Mage`
- `DMG_Ranger`
- `DMG_Assassin`
- `DMG_Priest`

E.g.
>Alice has 1000 max HP and 0% Heal_Received (no buff), then she receives the buffs `Heal_Received +10%` and `Heal_Received +30%`. Her Heal_Received will now be multiplied by `1.1 * 1.3 = 1.43`.  
>
>Bob has 0% Heal_Amount, then he successively receives the buffs `Heal_Amount +10%`, `Heal_Amount +20%`. His Heal_Amount will now be multiplied by `1.1 * 1.2 = 1.21`. 
>
>Now if Bob heals Alice with 15% of Alice's max HP, the final HP amount to be recovered by Alice is calculated as below:

In [2]:
from math import floor

Heal_Amount = floor(1000 * 0.15)
Heal_Amount = floor(Heal_Amount * 1.21)
Heal_Amount = floor(Heal_Amount * 1.43)

print(Heal_Amount) # 258, note that this value will be bounded by Alice's max HP minus Alice's current HP.

258


<a id='MIXED'></a>
### Mixed rule
The three base stats are using both additive and multiplicative rules:
- `Attack`
- `HP`
- `Armor`

For these three buffs, the additive part and the multiplicative part are calculated separately, e.g.  
>A girl initially has 100 HP, then she successively receives the buffs below:
>1. `HP +100`
>1. `HP +10%`
>1. `HP +20`
>1. `HP +30`
>1. `HP +30%`
>1. `HP +30%`
>
>Her final HP is given by the code below:

In [3]:
from math import floor

HP = floor(100)

for HP_ADD in [100, 20, 30]:
    HP += floor(HP_ADD)

for HP_MUL in [0.1, 0.3, 0.3]:
    HP = floor(HP * (1 + HP_MUL))

print(HP) # 464

464


<a id='BOUND'></a>
### Bounds of stats

Some stats have upper/lower bounds. See the [in-game Help](#HELP) for details.

Note that the bounds are applied when the corresponding stats are being used instead of being assigned. E.g.
>A girl initially has 100% Armor_Brk, then she receives a buff that adds 20% Armor_Brk and a debuff that reduces 30% Armor_Brk.  
>Her Armor_Brk now is 90% instead of 70%.

<a id='ORDER'></a>
## Order of buffs

<a id='FORMULA'></a>
## Damage formula

<a id='HELP'></a>
## Battle (in-game Help)

### Battle Line-Up
Girls positioned in the Front Row will take more damage than Girls in the Back Row. Adjust the Girls' positions according to their Class so that the whole team survives longer.

### Faction Rivalry
Faction Rivalry gives boosts to Girls when fighting against specific Factions. Deploy Girls based on the boosts they will gain from fighting against your opponent's Faction.

### Faction Aura
Certain Faction combinations will give Aura boosts to all of your Girls in battle.

### Victory or Defeat
Defeat all your opponents' Girls within 15 Rounds for a Victory. Failure to finish within 15 rounds will result in a Defeat.

### Attack
Attack refers to the damage dealt to enemies. The higher Attack a Girl has, the more damage she will deal in an attack.

### HP
HP allows Girls to survive longer in Battle.

### Armor
Armor helps to reduce damage from your opponents.

### Speed
Speed determines the order that Girls attack in battle. Higher Speed allows Girls to attack first every round. If girls on the same team have equal Speed, the girl closest to the front first position will move first.

### Extra Skill Damage
Extra Skill Damage gives bonus damage to skill attacks, boosts the damage rate that is based on the characters' Attack attribute in Active skills only.

### Block
Block determines the probability of blocking an attack. Blocking an attack successfully will reduce 30% of the damage taken; if the attack was a critical strike, blocking will reduce 50% of the damage taken. Only damage based on the Attack of the attacker will be reduced by blocking.

### Precision
Precision lowers the probability of the target blocking an attack. Furthermore, every 1% of Precision will grant an extra 0.3% of damage. The extra damage will max out at 45%. Only damage based on the Attack of the attacker will be increased.

### Crit Resist
Reduce the chance to take Critical hits, the higher a girl's Crit Resist is, the less likely she's going to take a Critical hit.

### Crit Rate
Crit Rate determines the probability of performing a critical strike against an opponent, dealing significantly increased damage. Without extra Crit damage increases, a Critical Strike will cause 150% damage of a normal attack. Passive skills will not trigger Crit attacks.

### Crit Damage
Crit Damage determines how much extra damage is dealt when performing a critical strike. The highest Crit Damage that an attack can have is 150%.

### Armor Break
Armor Break decreases your opponents' Armor, allowing you to deal more damage to the opponent. The maximum amount of Armor Break that can be reached is 100%.

### Immunity
Immunity helps against attacks that restrict your Girls, like Petrify, Freeze, and Stun. Immunity is calculated differently from the control effect chances: for example, an attack with a 50% chance to restrict a girl may be possible to control a girl with 70% Immunity.

### Control Rate
Increases the rate the enemy will be affected by your control skills. This status will be calculated together with the target's Immunity.

### Damage Reduction
Damage Reduction reduces the amount of damage inflicted by opponents, helping your Girls to survive longer. It will max out at 70%, and only works on damage based on the Attack of the attacker.

### Holy Damage
Holy Damage will not be affected by the armor of the target, but will be affected by Damage Reduction.

### Damage Depletion
Reduces all damages received and is calculated separately from Damage Reduction. 70% Damage Depletion maximum is obtainable by a Girl.

### Dodge
Provides Girls chance to avoid damages from enemies' Active Skills and Basic Attacks.(Servant's Active skill cannot be dodged)

### Energy
Every girl starts with 50 Energy when the battle begins. A girl will gain 50 Energy when she attacks the enemy, and will gain 10 Energy when she is attacked. If she takes a Critical strike, she will gain 20 Energy. A girl can only use her active skill when she has 100 or more Energy. When she has more than 100 Energy, every extra Energy will be counted as 1% of Extra Skill Damage.

### The color of DMG
If it's white, it means this attack doesn't trigger Crit attack and the attacker doesn't have Faction Rivalry buff. If it's yellow, it means this attack doesn't trigger Crit attack and the attacker is buffed by Faction Rivalry. If it's red, it means this attack is a Crit attack (the attack can be either buffed by Faction Rivalry or not). And if it's green, it's showing the healing amount.

### Skill Effect
Buffs or debuffs labeled with “Skill effect” cannot be removed by Potential or Skills.